# Static Dashboard Developing
### Run on Jupyter Notebook

### Install necessary packages for creating interactive dashboard 
- panel
- hvplot
- vega_datasets
- plotly
- seaborn

### Import libraries 

In [33]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

from vega_datasets import data as vds
import holoviews as hv
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots

### Upload and read csv file 

In [34]:
happiness = pd.read_csv('Worldhappinessreport2020-2022.csv')
happiness

,Year,RANK,Country name,Regional indicator,Happiness score,Whisker-high,Whisker-low,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,2020,1,Finland,Western Europe,7.8087,7.869766,7.747634,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,2020,2,Denmark,Western Europe,7.6456,7.711245,7.579955,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,2020,3,Switzerland,Western Europe,7.5599,7.628528,7.491272,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,2020,4,Iceland,Western Europe,7.5045,7.621347,7.387653,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,2020,5,Norway,Western Europe,7.4880,7.556281,7.419719,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2022,139,Tanzania,Sub-Saharan Africa,3.7020,3.847000,3.558000,0.848000,0.597000,0.425000,0.578000,0.248000,0.270000,0.735000
423,2022,140,Sierra Leone,Sub-Saharan Africa,3.5740,3.732000,3.416000,0.686000,0.416000,0.273000,0.387000,0.202000,0.055000,1.556000
424,2022,144,Zimbabwe,Sub-Saharan Africa,2.9950,3.110000,2.880000,0.947000,0.690000,0.270000,0.329000,0.106000,0.105000,0.548000
425,2022,145,Lebanon,Middle East and North Africa,2.9550,3.049000,2.862000,1.392000,0.498000,0.631000,0.103000,0.082000,0.034000,0.216000


In [35]:
# cache data to improve dashboard performance
if 'data1' not in pn.state.cache.keys():

    df1 = pd.read_csv('Worldhappinessreport2020-2022.csv')

    pn.state.cache['data1'] = df1.copy()

else: 

    df1 = pn.state.cache['data1']

In [36]:
# Make DataFrame Pipeline Interactive
idf1 = df1.interactive()

### Create widgets

In [37]:
#year slider
year_slider = pn.widgets.IntSlider(name='Year slider', start=2020, end=2022, step=1, value=2022)
year_slider

IntSlider(end=2022, name='Year slider', start=2020, value=2022)

In [38]:
idf1.columns

In [39]:
# Radio buttons group
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=[
        'Happiness score',
      'Log GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Dystopia + residual'
    ],
    button_type='success'
)

In [40]:
# Controlor
pn.Row(yaxis_co2.controls(jslink=True), yaxis_co2)

Row
    [0] Tabs
        [0] WidgetBox(name='Controls', width=300)
            [0] StaticText(value='<b>Controls</b>')
            [1] Checkbox(name='Disabled')
            [2] Select(name='Button type', options=OrderedDict([('default', ...]), value='success')
            [3] LiteralInput(name='Options', serializer='json', value=['Happiness score', ...])
            [4] LiteralInput(name='Value', serializer='json', value='Happiness score')
            [5] Select(name='Orientation', options=OrderedDict([('horizontal'...]), value='horizontal')
        [1] WidgetBox(name='Layout', width=300)
            [0] StaticText(value='<b>Layout</b>')
            [1] LiteralInput(name='Align', serializer='json', value='start')
            [2] LiteralInput(name='Aspect ratio', serializer='json')
            [3] LiteralInput(name='Background', serializer='json')
            [4] ListInput(name='Css classes', serializer='json', type=<class 'list'>)
            [5] IntInput(name='Width', start=0)
            [6] IntInput(name='Height', start=0)
            [7] IntInput(name='Min width', start=0)
            [8] IntInput(name='Min height', start=0)
            [9] IntInput(name='Max width', start=0)
            [10] IntInput(name='Max height', start=0)
            [11] LiteralInput(name='Margin', serializer='json', value=(5, 10))
            [12] Select(name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [13] Select(name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(name='Sizing mode', options=OrderedDict([('fixed', ...]))
            [15] Checkbox(name='Visible', value=True)
            [16] Checkbox(name='Loading')
    [1] RadioButtonGroup(button_type='success', name='Y axis', options=['Happiness score', ...], value='Happiness score')

### 1 Dashboard: Sortable table

In [41]:
table_pipeline = (
    idf1[
        (idf1.Year <= year_slider)
    ]
    .groupby(['RANK','Year','Country name', 'Regional indicator'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year') 
    .reset_index(drop=True)
)
table = table_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
table

### 2 Dashboard: Line graph

In [42]:
#sort out the focusing countries
hy1 = df1.loc[df1['Country name'] == 'Finland']
hy2 = df1.loc[df1['Country name'] == 'Denmark']
hy3 = df1.loc[df1['Country name'] == 'Switzerland']
hy4 = df1.loc[df1['Country name'] == 'Afghanistan']
hy5 = df1.loc[df1['Country name'] == 'Lebanon']
hy6 = df1.loc[df1['Country name'] == 'Jordan']

In [43]:
#combine datafream to create new datafream
##easy to compare
hy_combined1 = pd.concat([hy1,hy2,hy3])
hy_combined1

,Year,RANK,Country name,Regional indicator,Happiness score,Whisker-high,Whisker-low,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,2020,1,Finland,Western Europe,7.8087,7.869766,7.747634,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
153,2021,1,Finland,Western Europe,7.8420,7.904000,7.780000,1.446000,1.106000,0.741000,0.691000,0.124000,0.481000,3.253000
302,2022,1,Finland,Western Europe,7.8210,7.886000,7.756000,1.892000,1.258000,0.775000,0.736000,0.109000,0.534000,2.518000
1,2020,2,Denmark,Western Europe,7.6456,7.711245,7.579955,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
154,2021,2,Denmark,Western Europe,7.6200,7.687000,7.552000,1.502000,1.108000,0.763000,0.686000,0.208000,0.485000,2.868000
303,2022,2,Denmark,Western Europe,7.6360,7.710000,7.563000,1.953000,1.243000,0.777000,0.719000,0.188000,0.532000,2.226000
2,2020,3,Switzerland,Western Europe,7.5599,7.628528,7.491272,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
155,2021,3,Switzerland,Western Europe,7.5710,7.643000,7.500000,1.566000,1.079000,0.816000,0.653000,0.204000,0.413000,2.839000
305,2022,4,Switzerland,Western Europe,7.5120,7.586000,7.437000,2.026000,1.226000,0.822000,0.677000,0.147000,0.461000,2.153000


In [44]:
#make the datafream interactive
hyc1 = hy_combined1.interactive()

#create pipeline
hyc1_pipeline = (
    hyc1[
        (hyc1.Year <= year_slider)
    ]
    .groupby(['Country name', 'Year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year') 
    .reset_index(drop=True)
)
#hyc1_pipeline

In [45]:
#plot line graph with widgets
line1_plot = hyc1_pipeline.hvplot(x = 'Year', by='Country name', y=yaxis_co2,line_width=2, title="Top 3 Countries")
line1_plot

In [46]:
#combine datafream to create new datafream
hy_combined2 = pd.concat([hy4,hy5,hy6])
hy_combined2

,Year,RANK,Country name,Regional indicator,Happiness score,Whisker-high,Whisker-low,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
152,2020,153,Afghanistan,South Asia,2.5669,2.628270,2.505530,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226,1.507236
301,2021,149,Afghanistan,South Asia,2.5230,2.596000,2.449000,0.370000,0.000000,0.126000,0.000000,0.122000,0.010000,1.895000
426,2022,146,Afghanistan,South Asia,2.4040,2.469000,2.339000,0.758000,0.000000,0.289000,0.000000,0.089000,0.005000,1.263000
110,2020,111,Lebanon,Middle East and North Africa,4.7715,4.857258,4.685742,0.889233,1.192493,0.788671,0.185517,0.158524,0.021518,1.535542
275,2021,123,Lebanon,Middle East and North Africa,4.5840,4.691000,4.477000,1.045000,0.868000,0.595000,0.175000,0.140000,0.026000,1.736000
425,2022,145,Lebanon,Middle East and North Africa,2.9550,3.049000,2.862000,1.392000,0.498000,0.631000,0.103000,0.082000,0.034000,0.216000
118,2020,119,Jordan,Middle East and North Africa,4.6334,4.748552,4.518248,0.785179,1.140119,0.777625,0.424855,0.091495,0.151879,1.262258
279,2021,127,Jordan,Middle East and North Africa,4.3950,4.516000,4.273000,0.890000,0.685000,0.583000,0.455000,0.079000,0.150000,1.553000
417,2022,134,Jordan,Middle East and North Africa,4.1520,4.276000,4.029000,1.324000,0.724000,0.675000,0.476000,0.058000,0.200000,0.695000


In [47]:
# make dataframe interactive
hyc2 = hy_combined2.interactive()

#create pipeline
hyc2_pipeline = (
    hyc2[
        (hyc2.Year <= year_slider)
    ]
    .groupby(['Country name', 'Year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year') 
    .reset_index(drop=True)
)

#hyc2_pipeline

In [48]:
#creat a line graph fromnew dataframe
line2_plot = hyc2_pipeline.hvplot(x = 'Year', by='Country name', y=yaxis_co2,line_width=2, title="On Developing Countries")
line2_plot

In [49]:
#combine datafream to create new datafream
hy_combined3 = pd.concat([hy1,hy2,hy3,hy4,hy5,hy6])
hy_combined3

,Year,RANK,Country name,Regional indicator,Happiness score,Whisker-high,Whisker-low,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,2020,1,Finland,Western Europe,7.8087,7.869766,7.747634,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
153,2021,1,Finland,Western Europe,7.8420,7.904000,7.780000,1.446000,1.106000,0.741000,0.691000,0.124000,0.481000,3.253000
302,2022,1,Finland,Western Europe,7.8210,7.886000,7.756000,1.892000,1.258000,0.775000,0.736000,0.109000,0.534000,2.518000
1,2020,2,Denmark,Western Europe,7.6456,7.711245,7.579955,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
154,2021,2,Denmark,Western Europe,7.6200,7.687000,7.552000,1.502000,1.108000,0.763000,0.686000,0.208000,0.485000,2.868000
303,2022,2,Denmark,Western Europe,7.6360,7.710000,7.563000,1.953000,1.243000,0.777000,0.719000,0.188000,0.532000,2.226000
2,2020,3,Switzerland,Western Europe,7.5599,7.628528,7.491272,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
155,2021,3,Switzerland,Western Europe,7.5710,7.643000,7.500000,1.566000,1.079000,0.816000,0.653000,0.204000,0.413000,2.839000
305,2022,4,Switzerland,Western Europe,7.5120,7.586000,7.437000,2.026000,1.226000,0.822000,0.677000,0.147000,0.461000,2.153000
152,2020,153,Afghanistan,South Asia,2.5669,2.628270,2.505530,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226,1.507236


In [50]:
# make dataframe interactive
hyc3 = hy_combined3.interactive()

#create pipeline
hyc3_pipeline = (
    hyc3[
        (hyc3.Year <= year_slider)
    ]
    .groupby(['Country name', 'Year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year') 
    .reset_index(drop=True)
)

In [51]:
#create line graph from new data frame
line3_plot = hyc3_pipeline.hvplot(x = 'Year', by='Country name', y=yaxis_co2,line_width=2, title="Main Focusing Countries")
line3_plot

### 3 Dashboard: Pie chart

In [53]:
# Filter the years 2020-2022
years = [2020, 2021, 2022]
dfs = [df1[df1['Year'] == year] for year in years]

# Create the subplots
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}]*3], 
                    subplot_titles=[f" {year}" for year in years])

# Plot the pie charts for each year
for i, df in enumerate(dfs):
    df_agg = df.groupby('Regional indicator')['Happiness score'].mean().reset_index()
    fig.add_trace(px.pie(df_agg, values='Happiness score', names='Regional indicator').data[0], row=1, col=i+1)

# Update the layout
fig.update_layout(title='Happiness Score by Region', showlegend=False)

pie_chart=fig.show()
pie_chart

In [54]:
NA = df1.loc[df1['Regional indicator'] == 'North America and ANZ']
NA1 = NA.interactive()

In [55]:
NA1.hvplot(kind='table', width=1300)

### 4 Dashboard: Bar chart

In [56]:
# show data frame
hy_combined3

,Year,RANK,Country name,Regional indicator,Happiness score,Whisker-high,Whisker-low,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,2020,1,Finland,Western Europe,7.8087,7.869766,7.747634,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
153,2021,1,Finland,Western Europe,7.8420,7.904000,7.780000,1.446000,1.106000,0.741000,0.691000,0.124000,0.481000,3.253000
302,2022,1,Finland,Western Europe,7.8210,7.886000,7.756000,1.892000,1.258000,0.775000,0.736000,0.109000,0.534000,2.518000
1,2020,2,Denmark,Western Europe,7.6456,7.711245,7.579955,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
154,2021,2,Denmark,Western Europe,7.6200,7.687000,7.552000,1.502000,1.108000,0.763000,0.686000,0.208000,0.485000,2.868000
303,2022,2,Denmark,Western Europe,7.6360,7.710000,7.563000,1.953000,1.243000,0.777000,0.719000,0.188000,0.532000,2.226000
2,2020,3,Switzerland,Western Europe,7.5599,7.628528,7.491272,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
155,2021,3,Switzerland,Western Europe,7.5710,7.643000,7.500000,1.566000,1.079000,0.816000,0.653000,0.204000,0.413000,2.839000
305,2022,4,Switzerland,Western Europe,7.5120,7.586000,7.437000,2.026000,1.226000,0.822000,0.677000,0.147000,0.461000,2.153000
152,2020,153,Afghanistan,South Asia,2.5669,2.628270,2.505530,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226,1.507236


In [57]:
#create pipeline
hyc3_pipline = (
    hyc3[
        (hyc3.Year <= year_slider)
    ]
    .groupby(['Country name', 'Year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year') 
    .reset_index(drop=True)
)
#hyc3_pipline

In [58]:
# set color
custom_palette = sns.color_palette("Set2", 12)

# create bar chat
bar_chart=hyc3_pipline.hvplot(x = 'Year', by='Country name', y=yaxis_co2,kind='bar',rot=30, title="World Happiness", width=1000,color=custom_palette)
bar_chart

### 5 Dashboard: Scatter plot

In [59]:
scatter_pipeline = (
    idf1[
        (idf1.Year <= year_slider)
    ]
    .groupby(['RANK','Year','Country name'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='RANK') 
    .reset_index(drop=True)
)
scatter_pipeline

In [60]:
# create scatter plot
scatter = scatter_pipeline.hvplot(x='Country name', 
            y=yaxis_co2, 
            kind='scatter',
            c='Year',
            title = 'Happiness score in each countries 2020-2022',rot=70, height = 500)
scatter

In [61]:
scatter1 = idf1.hvplot(x='Country name', 
            y='Happiness score', 
            kind='scatter',
            c='Year',
            title = 'Happiness score in each countries 2020-2022',rot=70, height = 500)
scatter1

### Creating Dashboard


In [62]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World Happiness Report Dashboard', 
    sidebar=[pn.pane.Markdown("# World Happiness Report 2020-2022"),
             pn.pane.Markdown("## Year"),   
             year_slider,
             pn.pane.PNG('happy.png', sizing_mode='scale_both'),
             pn.pane.Markdown("#### The World Happiness Report is an annual report produced by the United Nations Sustainable Development Solutions Network. It ranks countries based on how happy their citizens perceive themselves to be, using various factors such as income, social support, freedom, trust, generosity, and health to measure happiness."), 
             ],
    main=[pn.Row(pn.Column(yaxis_co2,
                           line3_plot.panel(width=500), margin=(-20,25)), 
                 bar_chart.panel(width=500)), 
          pn.Row( pn.Column(scatter.panel(width=600), margin=(0,25)), 
                 pn.Column(table.panel(width=400)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
template.servable();

Launching server at http://localhost:50044
